# Data Simulation

This is short overview of undiscussed topics in the Master's Thesis. It covers some theoretical background on following topics:

1. **Structural/Shot noise**: How to measure the SNR and SSNR
2. **Radiation damage**: Calculation
3. **Detector**: Differences between direct electron detector and CCD detector 


## 1. Noise
The **true SNR** of an image can only be determined from simulated data. If the variance of the signal and noise are known, and the signal and noise are both zero-mean, SNR can be:

\begin{equation*}
\mathrm{SNR} = \frac{\sigma^2_\mathrm{signal}}{\sigma^2_\mathrm{noise}}.
\end{equation*}

So if I simulate a micrograph similar to the micrograph 004 from the VPP paper (EMPIAR-10078) with approximately the same number of particles, the SNR can be calculated from the variance of the signal of a micrograph without shot noise and the variance of a noise patch.

This can be done in two ways: either from a micrograph fully exposed to an electron dose of 3900 e/A² and no radiation damage, or by averaging a simulated frame stack with radiation damage. The results are:

|Description|SNR|
|:--------|:-----:|
|full exposure at 3900 e/A²|0.0886|
|average of 24 frames, 1,625 e/A² per frame, radiation damage|0.0682|
|average of 22 frames, 1,625 e/A² per frame, radiation damage|0.0675|
|mean SNR per frame (previous value divided by number of frames)|0.0031|



### 1.1 Measuring the SNR
There are two methods to calculate the SNR:

**Method 1**:
$var ( \overline p_{[N]} )$ is the variance of the average image, $var(p)$ is the variance of the signal, $var(n)$ is the variance of the noise.

\begin{equation*}
var(p_i) = var(p) + var(n) \\
var ( \overline p_{[N]} ) = var(p) + \frac{1}{N} \cdot var(n)
\end{equation*}

**Method 2**:
$\rho$ is the cross correlation coefficient  of two images of the same imaging field, SNR is ($\alpha$). This formula gives an overly optimistic estimation of the true SNR, but is ok for comparisons.

\begin{equation*}
\alpha = \frac{\rho}{1-\rho}
\end{equation*}

##### Links:
* [Three dimensional electron microscopy of molecular assemblies, Frank](http://www.oxfordscholarship.com/view/10.1093/acprof:oso/9780195182187.001.0001/acprof-9780195182187-chapter-2)
* [Determination of Signal-to-Noise Ratios and Spectral SNRs in Cryo-EM Low-dose Imaging of Molecules](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC2700974/)



#### 1.1.1 Shot noise
The amount of noise that is being contributed by shot noise can be determined by comparing two micrographs of the same imaging field. Since I did not have two successive micrographs of the same imaging field, I decided to make the calculations on a frame stack of 22 aligned frames. Method 1 gives the SNR of a single frame as output, Method 2 can be used both on caomparing two single frames as well as the average of frames, e.g. odd vs. even.

I also decided to compute the SNR of simulated images the same way as for the real images, for comparison. The images were simulated without structural noise, so the only noise source is shot noise. Radiation damage was taken into account. 

|SNR computing method|Real data|Simulated data|
|:-------------------|:-------:|:------------:|
|Method 1|0.001 to 0.003|stabilises at ca. 0.00249|
|Method 2|**11 frames**: 0.046 <br> **1 frame**: ~0,042|**11 frames**: 0,376 <br> **1 frame**: 0.034|

#### 1.1.2 Structural noise
The structural noise has both a constant and a variable component throughout a movie stack. The constant component comes with the amorphous structure of the ice and structural variations of the protein, the variable component can be reffered to as the effect of brownian motion.

The constant component can clearly be seen in the power spectra of the movie stacks. While thon rings are not visible in single frames, they appear in the average of aligned frames.

The amount of structural noise can be determined by comparing particles with the same orientation. In this case, using the first method for calculating the SNR is more appropiate. Also, using a mask for the calculation of the SNR would improve the estimation of the SNR of the particle alone. But one must not forget that the particle is embeded in ice. So one could also estimate the SNR of amorphous ice, calculate the ice thickness and estimate the SNR of the particle alone. Well, this would not be that easy. 

A very simple way to do this would be to just calculate the SNR of a single particle of an aligned movie stack by the first method. This SNR is the result of shot noise and all other noise sources like radiation damage, particle motion, structural variation etc., refered to as structural noise. Assuming the determined SNR is denoted as $\alpha_{comp\_struct}$ and the SNR related to the shot noise is $\alpha_{exp}$, then the value of the true structural SNR $\alpha_{true\_struct}$, untangled from the shot noise, is

\begin{equation*}
\alpha_{true\_struct} = \frac{1}{ \frac{(1+1/\alpha_{comp\_struct})}{ (1+1/\alpha_{exp})} - 1}
\end{equation*}

So the true structural SNR $\alpha_{true\_struct}$ should be the value achieved for a simulated image without the addition of shot noise.

\begin{equation*}
\alpha_{true\_struct} = \frac{var(signal)}{var(struct\_noise)}
\end{equation*}

#### 1.1.3 Ice thickness variations 
One must also consider the effect of low frequency noise on the accuracy of the alignment. Ice is generally not equally flat everywhere on the sample, so local variations influence the densities. Though such an effect is fairly easy to add to the simulations. 

### 1.2 SSNR
The spectral signal to noise ratio can be determined by following equation:

\begin{equation*}
SSNR(r) = \frac{ FSC(r) }{ 1 - FSC(r) }
\end{equation*}

The three-dimensional Fourier Shell Correlation (FSC) was introduced by Harauz and van Heel in 1986. It measures the normalised cross correlation coefficient between two 3D volumes over corresponding shells in Fourier space, i.e., as a function of spatial frequency: [ref](https://www.imagescience.de/fsc.html)

\begin{equation*}
FSC(r) = 
\frac{\displaystyle\sum_{r_i \in r}{ F_1(r_i) \cdot F_2(r_i)^{\ast} }}
{\displaystyle\sqrt[2]{\sum_{r_i \in r}{ \left|F_1(r_i)\right|^2} \cdot \sum_{r_i \in r}{\left|F_2(r_i)\right|^2}}}
\end{equation*}

These software packages provide FSC computing packages:
* IMAGIC (free) [(link)](https://www.imagescience.de/fsc.html)
* EMAN2 [(link)](https://groups.google.com/forum/#!topic/eman2/feRJkJU5S_E)
* Python script [(link)](https://github.com/s-sajid-ali/FRC)


## 2. Radiation damage
### 2.1 The exposure filter
The resolution dependent critical exposure is described in the Grigorieff paper for a Rotavirus at 300 kV as:

\begin{equation*}
N_e(k) = a \cdot k^b + c
\end{equation*}

where $k$ is the spatial frequency in units $1/\unicode{x212B}$. The critical exposure is the exposure at which the spot intensity is reduced to $1/e$. For the the Rotavirus at 300 kV the values determined are $a=0.245$, $b=-1.665$ and $c=2.81$. The frequency dependent SNR, also known as spectral SNR decays exponentially:

\begin{equation*}
SNR(k, N) = SNR(k, 0) \cdot exp\left(-\frac{N}{N_e(k)}\right)
\end{equation*}

The fourier amplitudes decay by the square root of the exponential decay of the SNR, so the amplitudes of the fourier transform decay by:

\begin{equation*}
A(k, N) = A(k, 0) \cdot exp\left(-\frac{N}{2 \cdot N_e(k)}\right)
\end{equation*}
### 2.2 Limitations
The parameters for the exposure filter are only determined for the Rotavirus at 300 kV. Different proteins and different voltage would give different critical exposures. Some structural features are more sensitive to radiation damage than others. For example carboxyl groups are highly affected by radiation damage, while aromatic groups are less sensitive. Also secondary structure features like the $\alpha$-helix backbone are very resistant to radiation damage. Measurements of critical exposure are also affected by alignment accuracy of the particles, which in turn is influenced by beam induced movement and rotation.
For more information please refer to Grigorieffs paper.

## 3. Detector

### 3.1 Direct electron detector

Described in _Vulovic et al. 2013 - Image formation modeling in cryo-electron microscopy_

\begin{equation*}
I(x,y)= \mathfrak{F}^{-1} \left [ \mathfrak{F} \left [ Poisson \left ( \Phi_e \cdot \mathfrak{F}^{-1} \left [ \tilde{I_0}(q) \sqrt{DQE(q))} \right ]  \right ) \right ] \cdot NTF(q) \right ]
\end{equation*}

\begin{equation*}
DQE(q)=\frac{MTF^2(q)}{NTF^2(q)}
\end{equation*}

### 3.2 CCD detector

Used in TEM-Simulator

\begin{equation*}
\mu(x,y)=Poisson(\Phi_e\cdot I_0(x,y))
\end{equation*}

\begin{equation*}
{I}'(x,y)=C_{gain}\cdot InvGauss \left (\mu(x,y), \sigma=\mu(x,y)\cdot\frac{1}{C_{DQE}-1} \right)
\end{equation*}

\begin{equation*}
I(x,y)=\mathfrak{F}^{-1}\left[\tilde{{I}'}(q) \cdot MTF(q) \right ]
\end{equation*}


# Critical exposure curve
The critical exposure curve is measured as follows:
* align movie frames
* select only movie frames that have constant movement
* create particle reconstructions of 3 movie frames each
* compute the spectral signal to noise ratio of the entire map as follows:

\begin{equation*}
SSNR(k) = \frac{2 \cdot FSC(k)}{1 - FSC(k)}
\end{equation*}

* for each map created at different exposures, read the SNR at each frequency and plot it against the cumulative exposure

<img src="https://prod--cdn-iiif.elifesciences.org/lax:06980/elife-06980-fig4-v2.tif/full/1500,/0/default.jpg" width="500" height="400" />

* the slopes describe the fading of the signal, which increases as the resolution gets higher

* the frequency dependent critical exposure $N_e(k)$ is the number of electrons/A² after which the intensity of the signal is reduced to 1/e of its starting value. 

* The SNR(k) after an exposure of N electrons/A² is

\begin{equation*}
SNR(k,N) = SNR(k,0) \cdot e^{-\frac{N}{N_e(k)}}
\end{equation*}

so the plot of $ln(SNR(k))$ vs N has a slope of $-1/N_e(k)$

* A function $y=ax^b+c$ can be fitted to the measured points, with $a=0.245$, $b=-1.665$ and $c=2.81$

<img src="https://prod--cdn-iiif.elifesciences.org/lax:06980/elife-06980-fig5-v2.tif/full/,1500/0/default.jpg" width="500" height="400" />

* the fading of the image Fourier amplitudes follows the square root of the exponential decay of the SNR

\begin{equation*}
A(k,N) = A(k,0) \cdot e^{-\frac{N}{2N_e(k)}}
\end{equation*}

so the amplitudes of the Fourier transform decay by

\begin{equation*}
A(k, N) = A(k, 0) \cdot exp\left(-\frac{N}{2 \cdot N_e(k)}\right) \\
N_e(k) = a \cdot k^b + c
\end{equation*}